# Human's emotion recognition using thermal images
## *TEACHER: NGUYEN VIET HUNG*
## *MEMBER*
- Vuong Le Minh Nguyen
- Luong Cong Tam

#### Import Library & Settings

In [ ]:
import pandas as pd
import sklearn
import scipy
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import random
import IPython.display as display
from configparser import ConfigParser
from time import sleep

np.set_printoptions(threshold=np.inf) # Setting for full numpy matrices printing

In [ ]:
# Load db config:
dbconfig = ConfigParser()
dbconfig.read('database.ini')

# Assign dbDir with local config:
dbDir = dbconfig['LOCAL']['location'] #Database directory
eDir = {
    "angry":"anger",
    "fear":"fear",
    "happy":"happy",
    "sad":"sadness",
    "neutral":"neutral"
} #Name of directory of each Emotion

#### Function definition

In [ ]:
def frameShow(frame, ):
    w, h = frame.shape
    _, ret = cv2.imencode('.jpg',frame)
    i = display.Image(data=ret, width = w*3, height = h*3)
    display.display(i)
    sleep(0.05)
    display.clear_output(True)

In [ ]:
def getSVXFiles(fd):
    return glob.glob(fd + '/*.svx') + glob.glob(fd + '/*.SVX')

In [ ]:
def readSVX(file, frame_limit = 6990):
    # Default frame_limit is based on the requirement of at least 1GB of free RAM (6990*153600B ~~ 1GB)
    # Open file to read:
    f = open(file,'rb') 
    
    # Skip 128 bytes from the file's beginning:
    f.seek(128) 
    
    # Calculate grayscale converter:
    [CA,CB] = np.divide(np.frombuffer(f.read(8), dtype=np.dtype('<i4'), count=2), 2**16)
    #Array from buffer
    #Data type: int32 (4 bytes/each - 'i4') 
    #Byte-order:  Litte-endian ('<')
    
    # Skip to the beginning of first frame to start reading:
    f.seek(1104) 
    
    list_img2d = []

    for i in range (0,frame_limit):
        buffer = f.read(153600)
        
        # Check EOF:
        if len(buffer) < 153600:
            break
        
        img = np.frombuffer(buffer,dtype=np.dtype('>i2'))
        #Array from buffer
        #Data type: int16 (2 bytes/each - 'i2') 
        #Byte-order:  Big-endian ('>')
        
        # Convert read img to grayscale (temperature) using converter, then reshape from 1D to 2D:
        img2d = (img * CB + CA).reshape(240,320)
        
        # Add read frame to the result array:
        list_img2d.append(img2d) 
        
#         frameShow(img2d)
#         getFace(img2d)
        
#         sleep(0.75)
    
        # Move to the beginning of next frame to continue reading:
        f.seek(2592,1) 
    
    f.close()
    return np.array(list_img2d)

In [ ]:
def getROI(frame):
    tMax = np.amax(frame)
    tMin = np.amin(frame)
    # TODO: Continue writing getROIfunction

In [ ]:
def getFace(frame, minTemp = 30.0, fitEllipsePoint = 20, faceBox = (220,180)):
    # TODO - getFace function
    xFrame, yFrame = frame.shape
    topEdge, rightEdge, bottomEdge, leftEdge = xFrame, 0, 0, yFrame
    ret = np.copy(frame) # Do not change anythings in input array
    
    for x in range(xFrame):
        for y in range(yFrame):
            if (ret[x][y] < minTemp):
                ret[x][y] = 5
            else:
                topEdge = min(topEdge, x)
                rightEdge = max(rightEdge, y)
                bottomEdge = max(bottomEdge, x)
                if x <= (topEdge + faceBox[0])/2:
                    leftEdge = min(leftEdge, y)            
                    
    
    
#     print(topEdge, rightEdge, bottomEdge, leftEdge)
    
    ret = ret[topEdge:topEdge+220, leftEdge:leftEdge+190]
#     plt.imshow(ret[topEdge:bottomEdge, leftEdge:rightEdge])
#     print(type(ret[0][0]))
    topEdge = max(topEdge - 10, 0)
    leftEdge = max(leftEdge -10, 0)
    
#     sliced = np.uint8(ret[topEdge:topEdge+faceBox[0], leftEdge:leftEdge+faceBox[1]])
    sliced = np.uint8(ret)

#     print(sliced[topEdge:topEdge+faceBox[0], leftEdge:leftEdge+faceBox[1]])
    blured = cv2.GaussianBlur(sliced.copy(),(5,5),1)
    cannied = cv2.Canny(blured,20,60)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(15,15))
    cannied = cv2.dilate(cannied, kernel)
    
    _, contours, hierarchy = cv2.findContours(cannied, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    drawedContours = cv2.drawContours(blured,contours,-1,(255,0,0), 1)
    
#     print(hierarchy)
#     print(contours)
    minEllipse = [None]*len(contours)
    
    for i, c in enumerate(contours):
#         print(i,c)
        if c.shape[0]>fitEllipsePoint and hierarchy[0, i, 3] == -1:
            minEllipse[i] = cv2.fitEllipse(c)
            
    drawing = np.zeros((sliced.shape[0], sliced.shape[1], 3), dtype=np.uint8)
    mask = drawing.copy()
    for i, c in enumerate(contours):
        color = (random.randint(0,256), random.randint(0,256), random.randint(0,256))
#         # contour
#         cv2.drawContours(drawing, contours, i, color)
        # ellipse
        if c.shape[0]>fitEllipsePoint and hierarchy[0, i, 3] == -1:
            cv2.ellipse(drawing, minEllipse[i], color, -1)
            cv2.ellipse(mask, minEllipse[i], color=(255,255,255), thickness = -1)
            
#     print(ret.shape, mask[:, :, 0].shape)
    
    mask = cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY)
    
    mask = np.float64(mask)
    
    print(mask)
        
    cropped = cv2.bitwise_and(ret, mask)
    
#     print(cropped)
    
    plt.subplot(121)
#     plt.imshow(ret[topEdge:topEdge+faceBox[0], leftEdge:leftEdge+faceBox[1]])
    plt.imshow(ret)

    plt.subplot(122)
    plt.imshow(cannied)
    
    plt.show()
    
    plt.subplot(121)
    plt.imshow(drawedContours)
    plt.subplot(122)
    plt.imshow(mask)
    
    plt.show()

In [ ]:
def frameCaRs(frame, width, height):
    # TODO - frame Crop and Resize function
    return 0

In [ ]:
# Demo reading imgs from SVX Files:
train = []
for file in getSVXFiles(dbDir+eDir['fear']):
    data = readSVX(file, 10)
    print(file, 'frames = ', len(data))
    train.extend(data)
    data = None

In [ ]:
for i in range(len(train)//10):
    getFace(train[i*10], 29.50, 100)